<b>This next part is for Ryan:</b> you might have to update the patch path

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage import io

In [11]:
patch_path = '/Users/abishek/Desktop/DataScienceClasses/TILseg/abi_patches/TCGA-3C-AALI-01Z-00-DX1_T370.tif'

In [12]:
patch_path[-4:]

'.tif'

In [13]:
model = KMeans(n_clusters=8, max_iter=20,
                  n_init=3, tol=1e-3)
# Reads the patch into a numpy uint8 array    
fit_patch = plt.imread(patch_path) 
# Linearizes the array for R, G, and B separately and normalizes
# The result is an N X 3 array where N=height*width of the patch in pixels
fit_patch_n = np.float32(fit_patch.reshape((-1, 3))/255.)
# Fits the model to our linearized and normalized patch data 
model.fit(fit_patch_n)

KMeans(max_iter=20, n_init=3, tol=0.001)

In [15]:
small_patch = fit_patch[:300, :400]

In [22]:
io.imsave('/Users/abishek/Desktop/DataScienceClasses/TILseg/abi_patches/test_patch.tif', small_patch)

In [32]:
# Predicting on the same patch we fit for now
pred_patch_n = fit_patch_n
labels = model.predict(pred_patch_n)

In [33]:
print(pred_patch_n.shape)
pred_patch_n

(12000000, 3)


array([[0.85490197, 0.7254902 , 0.8235294 ],
       [0.6784314 , 0.54901963, 0.64705884],
       [0.5882353 , 0.4627451 , 0.54901963],
       ...,
       [0.95686275, 0.8666667 , 0.9137255 ],
       [0.9607843 , 0.88235295, 0.92156863],
       [0.9411765 , 0.87058824, 0.9019608 ]], dtype=float32)

In [34]:
print(labels.shape)
labels

(12000000,)


array([4, 1, 3, ..., 0, 0, 0], dtype=int32)

In [35]:
fit_patch.shape[1]

4000

In [38]:
clustered_patch_labels = np.resize(labels, (fit_patch.shape[0], fit_patch.shape[1]))
clustered_patch_labels.shape

(3000, 4000)

Implementing <b>tilseg.cluster</b>

In [30]:
import cluster

In [31]:
model_T370 = cluster.cluster_model_fitter(
    patch_path='/Users/abishek/Desktop/DataScienceClasses/TILseg/abi_patches/TCGA-3C-AALI-01Z-00-DX1_T370.tif',
    algorithm='KMeans',
    n_clusters=4)

In [7]:
cluster.pred_and_cluster(
    model=model_T370,
    in_dir_path='/Users/abishek/Desktop/DataScienceClasses/TILseg/abi_patches',
    out_dir_path='/Users/abishek/Desktop/DataScienceClasses/Project/model_T370',
    )

In [4]:
model_super = cluster.cluster_model_fitter(
    patch_path='/Users/abishek/Desktop/DataScienceClasses/messing_around/super_patch.tif',
    algorithm='KMeans',
    n_clusters=4)

In [8]:
cluster.pred_and_cluster(
    model=model_super,
    in_dir_path='/Users/abishek/Desktop/DataScienceClasses/TILseg/abi_patches',
    out_dir_path='/Users/abishek/Desktop/DataScienceClasses/Project/model_super',
    )

You can check the output overlays created by both models to compare training on the same image versus training on the super patch.

In [32]:
ch_score_T370, db_score_T370 = cluster.clustering_score(
    model=model_T370, 
    patch_path='/Users/abishek/Desktop/DataScienceClasses/TILseg/abi_patches/TCGA-3C-AALI-01Z-00-DX1_T370.tif'
    )

print(ch_score_T370)
print(db_score_T370)

29723280.608573735
0.679668471385345


In [10]:
ch_score_super, db_score_super = cluster.clustering_score(
    model=model_super, 
    patch_path='/Users/abishek/Desktop/DataScienceClasses/TILseg/abi_patches/TCGA-3C-AALI-01Z-00-DX1_T370.tif'
    )

print(ch_score_super)
print(db_score_super)

25289272.250682004
0.6559003890987466


The Calinski-Harabasz Index is higher and better when fitting on the same patch, but the CH Index for fitting on the super patch is not too much lower. This means that the clusters are performing almost as good as the custom-fitted model.

The Davies-Bouldin Score is actually lower for the model fitted on the super patch. This means that the the clustering is actually better with fitting on the super patch according to this metric.

In [4]:
import matplotlib.pyplot as plt
import numpy as np

In [5]:
small_patch_path = '/Users/abishek/Desktop/DataScienceClasses/TILSeg/abi_patches/test_patch.tif'
small_patch = plt.imread(small_patch_path)
small_patch_n = np.float32(small_patch.reshape((-1, 3))/255.)

In [6]:
import sklearn.cluster

In [7]:
model_DBSCAN = sklearn.cluster.DBSCAN()

In [ ]:
# DBSCAN way too computationally expensive

# model_DBSCAN.fit(small_patch_n)

In [ ]:
model_OPT